In [1]:
import pandas as pd
import numpy as np

In [2]:
train_data = pd.read_csv('/home/guozijun/kaggle/Taxi Duration/data/train.csv')
test_data = pd.read_csv('/home/guozijun/kaggle/Taxi Duration/data/test.csv')

In [3]:
from datetime import datetime
train_data['pickup_time'] = train_data['pickup_datetime'].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
train_data['pickup_time_month'] = train_data['pickup_time'].map(lambda x: x.month)
train_data['pickup_time_weekday'] = train_data['pickup_time'].map(lambda x: x.weekday())
train_data['pickup_time_hour'] = train_data['pickup_time'].map(lambda x: x.hour)

In [4]:
train_data['store_and_fwd_flag'] = train_data['store_and_fwd_flag'].map(lambda x: 0 if x=='N' else 1)

In [5]:
#基于出现数量排序的函数
def sort_feature(df,s):
    s_counts = df[s].value_counts()
    s_dict = dict(zip(list(s_counts.index.values), range(1,len(s_counts)+1)))
    df[s+'_sort'] = df[s].map(s_dict)
    return df

In [6]:
#根据不同时间段用车次数排序
train_data = sort_feature(train_data,'pickup_time_month')
train_data = sort_feature(train_data,'pickup_time_weekday')
train_data = sort_feature(train_data,'pickup_time_hour')

In [7]:
#经纬度离散化
n=10
train_data['pickup_longitude_qcut'] = pd.qcut(train_data['pickup_longitude'],n,labels=range(n))
train_data['dropoff_longitude_qcut'] = pd.qcut(train_data['dropoff_longitude'],n,labels=range(n))
train_data['dropoff_latitude_qcut'] = pd.qcut(train_data['dropoff_latitude'],n,labels=range(n))
train_data['pickup_latitude_qcut'] = pd.qcut(train_data['pickup_latitude'],n,labels=range(n))
train_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_time_month,pickup_time_weekday,pickup_time_hour,pickup_time_month_sort,pickup_time_weekday_sort,pickup_time_hour_sort,pickup_longitude_qcut,dropoff_longitude_qcut,dropoff_latitude_qcut,pickup_latitude_qcut
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,...,3,0,17,1,7,6,4,7,6,7
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,...,6,6,0,5,6,18,5,1,2,2
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,...,1,1,11,6,5,12,5,0,0,6
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,...,4,2,19,2,4,2,0,0,0,0
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,...,3,5,13,1,2,10,6,6,8,9


In [10]:
train_data['trip_duration'].groupby(train_data['pickup_time_month']).mean().sort_values(ascending=False)

pickup_time_month
6    1013.367226
5     999.450555
4     962.891462
3     937.365363
1     922.373319
2     920.830453
Name: trip_duration, dtype: float64

In [7]:
#根据不同时间用车时长大小排序
month_count = train_data['trip_duration'].groupby(train_data['pickup_time_month']).mean().sort_values(ascending=False)
month_dict = dict(zip(list(month_count.index.values), range(1,len(month_count)+1)))
train_data['month_sort'] = train_data['pickup_time_month'].map(month_dict)
week_count = train_data['trip_duration'].groupby(train_data['pickup_time_weekday']).mean().sort_values(ascending=False)
week_dict = dict(zip(list(week_count.index.values), range(1,len(week_count)+1)))
train_data['week_sort'] = train_data['pickup_time_weekday'].map(week_dict)
hour_count = train_data['trip_duration'].groupby(train_data['pickup_time_hour']).mean().sort_values(ascending=False)
hour_dict = dict(zip(list(hour_count.index.values), range(1,len(hour_count)+1)))
train_data['hour_sort'] = train_data['pickup_time_hour'].map(hour_dict)

In [9]:
train_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,...,pickup_time_month_sort,pickup_time_weekday_sort,pickup_time_hour_sort,pickup_longitude_qcut,dropoff_longitude_qcut,dropoff_latitude_qcut,pickup_latitude_qcut,month_sort,week_sort,hour_sort
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,...,1,7,6,4,7,6,7,4,7,5
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,...,5,6,18,5,1,2,2,1,6,10
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,...,6,5,12,5,0,0,6,5,3,9
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,...,2,4,2,0,0,0,0,3,4,17
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,...,1,2,10,6,6,8,9,4,5,4


In [3]:
def Lon_Lat_distance(lon1,lat1,lon2,lat2):
    deltaLon = np.abs(lon1 - lon2)
    deltaLat = np.abs(lat1 - lat2)
    EARTH_RADIUS = 6371
    disLon,disLat = map(lambda x:np.pi*x*EARTH_RADIUS/180,(deltaLon,deltaLat))
    return disLon, disLat

def calcu_distance(lon1,lat1,lon2,lat2):
    disLon,disLat = Lon_Lat_distance(lon1,lat1,lon2,lat2)
    mht_distance = disLon + disLat
    line_distance = np.sqrt(disLon**2 + disLat**2)
    return mht_distance, line_distance

In [ ]:
# 两种距离特征（+2）
train_data['mht_distance'],train_data['line_distance'] = calcu_distance(train_data['pickup_longitude'],train_data['pickup_latitude'],
                                                                  train_data['dropoff_longitude'],train_data['dropoff_latitude'])

In [ ]:
train_data['mht_speed'] = 1000 * train_data['mht_distance'] / train_data['trip_duration']
train_data['line_speed'] = 1000 * train_data['line_distance'] / train_data['trip_duration']

In [ ]:
mht_region_speed = train_data.groupby(['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed.rename(columns={'mht_speed':'mht_region_speed'},inplace=True)
mht_region_speed_m = train_data.groupby(['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed_m.rename(columns={'mht_speed':'mht_region_speed_m'},inplace=True)
mht_region_speed_w = train_data.groupby(['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed_w.rename(columns={'mht_speed':'mht_region_speed_w'},inplace=True)
mht_region_speed_h = train_data.groupby(['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['mht_speed'].mean()
mht_region_speed_h.rename(columns={'mht_speed':'mht_region_speed_h'},inplace=True)

line_region_speed = train_data.groupby(['pickup_longitude_qcut','pickup_latitude_qcut',
                                                      'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed.rename(columns={'line_speed':'line_region_speed'},inplace=True)
line_region_speed_m = train_data.groupby(['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed_m.rename(columns={'line_speed':'line_region_speed_m'},inplace=True)
line_region_speed_w = train_data.groupby(['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed_w.rename(columns={'line_speed':'line_region_speed_w'},inplace=True)
line_region_speed_h = train_data.groupby(['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'],as_index=False)['line_speed'].mean()
line_region_speed_h.rename(columns={'line_speed':'line_region_speed_h'},inplace=True)

In [ ]:
train_data = pd.merge(train_data, mht_region_speed, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, mht_region_speed_m, how='left', on = ['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, mht_region_speed_w, how='left', on = ['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, mht_region_speed_h, how='left', on = ['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])

train_data = pd.merge(train_data, line_region_speed, how='left', on = ['pickup_longitude_qcut','pickup_latitude_qcut',
                                                     'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, line_region_speed_m, how='left', on = ['pickup_month','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, line_region_speed_w, how='left', on = ['pickup_weekday','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])
train_data = pd.merge(train_data, line_region_speed_h, how='left', on = ['pickup_hour','pickup_longitude_qcut','pickup_latitude_qcut',
                                                       'dropoff_longitude_qcut','dropoff_latitude_qcut'])

In [ ]:
mht_speed_m = train_data.groupby(['pickup_month'],as_index=False)['mht_speed'].mean()
mht_speed_m.rename(columns={'mht_speed':'mht_speed_m'},inplace=True)
mht_speed_w = train_data.groupby(['pickup_weekday'],as_index=False)['mht_speed'].mean()
mht_speed_w.rename(columns={'mht_speed':'mht_speed_w'},inplace=True)
mht_speed_h = train_data.groupby(['pickup_hour'],as_index=False)['mht_speed'].mean()
mht_speed_h.rename(columns={'mht_speed':'mht_speed_h'},inplace=True)

line_speed_m = train_data.groupby(['pickup_month'],as_index=False)['line_speed'].mean()
line_speed_m.rename(columns={'line_speed':'line_speed_m'},inplace=True)
line_speed_w = train_data.groupby(['pickup_weekday'],as_index=False)['line_speed'].mean()
line_speed_w.rename(columns={'line_speed':'line_speed_w'},inplace=True)
line_speed_h = train_data.groupby(['pickup_hour'],as_index=False)['line_speed'].mean()
line_speed_h.rename(columns={'line_speed':'line_speed_h'},inplace=True)

In [ ]:
train_data = pd.merge(train_data, mht_speed_m, how='left', on = ['pickup_month'])
train_data = pd.merge(train_data, mht_speed_w, how='left', on = ['pickup_weekday'])
train_data = pd.merge(train_data, mht_speed_h, how='left', on = ['pickup_hour'])

train_data = pd.merge(train_data, line_speed_m, how='left', on = ['pickup_month'])
train_data = pd.merge(train_data, line_speed_w, how='left', on = ['pickup_weekday'])
train_data = pd.merge(train_data, line_speed_h, how='left', on = ['pickup_hour'])

In [ ]:
train_data['longitude_delta'] = train_data['dropoff_longitude'] - train_data['pickup_longitude']
train_data['latitude_delta'] = train_data['dropoff_latitude'] - train_data['pickup_latitude']
train_data['longitude_dct'] = (train_data.longitude_delta > 0)*1
train_data['latitude_dct'] = (train_data.latitude_delta > 0)*1

In [ ]:
# 月平均出行时长，日平均出行时长，周平均出行时长
trip_duration_m = train_data.groupby(['pickup_month'],as_index=False)['trip_duration'].mean()
trip_duration_m.rename(columns={'trip_duration':'trip_duration_m'},inplace=True)
trip_duration_w = train_data.groupby(['pickup_weekday'],as_index=False)['trip_duration'].mean()
trip_duration_w.rename(columns={'trip_duration':'trip_duration_w'},inplace=True)
trip_duration_h = train_data.groupby(['pickup_hour'],as_index=False)['trip_duration'].mean()
trip_duration_h.rename(columns={'trip_duration':'trip_duration_h'},inplace=True)

In [ ]:
train_data = pd.merge(train_data, trip_duration_m, how='left', on = ['pickup_month'])
train_data = pd.merge(train_data, trip_duration_w, how='left', on = ['pickup_weekday'])
train_data = pd.merge(train_data, trip_duration_h, how='left', on = ['pickup_hour'])

In [11]:
train_data.to_csv('train_data_lgc_1.csv',index=False)

In [2]:
train_data = pd.read_csv('train_data_lgc_1.csv')